## 5. Losses, optimization, and initialization

In [1]:
"""
    Initialization
"""


import time
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as Func
from torch import nn
from torch import Tensor
from torchvision import datasets
from torch.autograd import Variable

### 1. Cross-entropy
- Mean-Squared Error is not the best loss function for Classification due to **conceptually wrong** $\rightarrow$ Cross-entropy (kinda better choice)
- 
